In [ ]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime
import time

In [6]:
import time
import datetime
s = "04/12/2020"
str(int(time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())))


'1607029200'

In [7]:
"""# Update your Search Settings here"""

#Create your timestamps and queries for your search URL
#https://www.unixtimestamp.com/index.php > Use this to create your timestamps



after = '05/03/2020'
before = '10/05/2022'


query = "GME|gamestop" #Keyword(s) to look for in submissions
sub = "wallstreetbets" #Which Subreddit to search in

after = str(int(time.mktime(datetime.datetime.strptime(after, "%d/%m/%Y").timetuple()))) #Submissions after this timestamp 
before = str(int(time.mktime(datetime.datetime.strptime(before, "%d/%m/%Y").timetuple()))) #Submissions before this timestamp




In [ ]:
def getPushshiftData(query, after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

In [ ]:
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    if 'selftext' in subm.keys():
        selftext = subm['selftext']
    else:
        selftext = " "
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,selftext,permalink,flair))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

In [ ]:

#subCount tracks the no. of total submissions we collect
subCount = 0
subStats = {}

# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(query, after, before, sub)


while len(data) > 1: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    print(after)
    #time.sleep(5)
    while True:
        try:
            data = getPushshiftData(query, after, before, sub) 
            break
        except ValueError:
            print("HELP ME PLEASE!!!!!!!!!!!!!!!!!!!!!!!!!!")
            continue  # ignore

In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))


In [ ]:
def updateSubs_file(attempt):
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    filename =str('comments_') + attempt+'.csv' #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","selftext","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")



In [ ]:
updateSubs_file('first')